In [2]:
%matplotlib inline
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import json
from apikey import api_key

**MLS Reading, Cleaning, and Output**

In [3]:
#Read mls data
summary_reader = os.path.join('..', 'resources','formatted - miniSummary.csv')

actSummary_data = pd.read_csv(summary_reader)

actSummary_data.head()

,Year,Property Type,yrTot_avail,yr_avg_avail,yrAvg_avail_all,yrTot_numSales,yrAvg_numSales,yrTot_spTrend,yrAvg_spTrend
0,2009,Private,NaN,NaN,NaN,NaN,NaN,1557195.0,129766.0
1,2010,Private,NaN,NaN,NaN,NaN,NaN,2819293.0,105175.0
2,2011,Private,NaN,NaN,NaN,NaN,NaN,2598148.0,111338.0
3,2012,Private,NaN,NaN,NaN,NaN,NaN,2824350.0,124025.0
4,2013,Private,NaN,NaN,NaN,18517.0,1543.0,3358988.0,155891.0


In [4]:
#trim years to 2014-2018
years=[int(2014), int(2015), int(2016), int(2017)]

#removes years not being measured
summaryAll = actSummary_data[actSummary_data["Year"].isin(years)]


summaryAll = summaryAll.reset_index(drop=True)
summaryAll.head()

,Year,Property Type,yrTot_avail,yr_avg_avail,yrAvg_avail_all,yrTot_numSales,yrAvg_numSales,yrTot_spTrend,yrAvg_spTrend
0,2014,Private,80137.0,6678.0,4095.0,15863.0,1322.0,3979052.0,175697.0
1,2015,Private,85971.0,7164.0,4189.0,21544.0,1795.0,4390349.0,190165.0
2,2016,Private,75308.0,6276.0,3763.0,21328.0,1777.0,4825235.0,211938.0
3,2017,Private,66236.0,5520.0,3400.0,21824.0,1819.0,5354151.0,234242.0
4,2014,Rental,18141.0,1512.0,NaN,9191.0,766.0,57154.0,2357.0


In [5]:
#Data separated by property type
rentalTotalData=summaryAll[summaryAll['Property Type']=='Rental']
privateTotalData=summaryAll[summaryAll['Property Type']=='Private']
allTotalData=summaryAll[summaryAll['Property Type']=='Both']
pAndR=summaryAll[summaryAll['Property Type']!='Both']

#Merge dataframes
availHsgDF = pd.merge(rentalTotalData,privateTotalData, on="Year")
availHsgDF = pd.merge(availHsgDF,allTotalData, on="Year")

In [6]:
#ename columns
availHsgDF.rename(columns={
                           'yrTot_avail_x': 'Rental Housing Available', 
                            'yrTot_avail_y': 'Private Housing Available', 
                           'yrTot_avail': 'Both Available Housing', 

                           'yrTot_numSales_x': 'Rentals Sales Number Total', 
                           'yrTot_numSales_y': 'Private Sales Number Total', 
                           'yrTot_numSales': 'Both Housing Sales Number Total', 

                           'yrTot_spTrend_x': 'Rentals Sales Price Trend', 
                           'yrTot_spTrend_y': 'Private Sales Price Trend Total', 
                           'yrTot_spTrend': 'Both Sales Price Trend', 
                         }, inplace=True)

#Drop duplicate & unnecessary columns
availHsgDF = availHsgDF.drop(columns=['Property Type_x', 'yr_avg_avail_x', 'yr_avg_avail_y', 'Property Type', 'yr_avg_avail','yrAvg_numSales', 'yrAvg_spTrend', 'yrAvg_avail_all_x',  'yrAvg_avail_all_y', 'yrAvg_spTrend_x', 'yrAvg_numSales_x', 'Property Type_y', 'yrAvg_avail_all_y',  'yrAvg_avail_all', 'yrAvg_spTrend_y', 'yrAvg_numSales_y'])

#Set index to year
availHsgDF = availHsgDF.set_index("Year")
availHsgDF.head()

,Rental Housing Available,Rentals Sales Number Total,Rentals Sales Price Trend,Private Housing Available,Private Sales Number Total,Private Sales Price Trend Total,Both Available Housing,Both Housing Sales Number Total,Both Sales Price Trend
Year,,,,,,,,,
2014,18141.0,9191.0,57154.0,80137.0,15863.0,3979052.0,98278.0,25054.0,4036206.0
2015,14561.0,8711.0,56901.0,85971.0,21544.0,4390349.0,100532.0,30255.0,4447250.0
2016,15008.0,8540.0,57055.0,75308.0,21328.0,4825235.0,90316.0,29868.0,4882290.0
2017,15375.0,8990.0,56868.0,66236.0,21824.0,5354151.0,81611.0,30814.0,5411019.0


In [9]:
#Output to .csv for future reference
outputfile = "../Resources/mlscleaned.csv"
availHsgDF.to_csv(outputfile)

PermissionError: [Errno 13] Permission denied: '../Resources/mlscleaned.csv'

**Census Pull as well as Reading, Cleaning, and Output**

In [18]:
#Create base urls
baseurl = "https://api.census.gov/data/"

#Create suffix for url
suffix = "&for=county:095&in=state:12&key=" + api_key

In [19]:
#Import csv
path = "../Resources/api_pulls.csv"
file = pd.read_csv(path)

#Splits the column names by the "_" delimiter to sort year easier, makes array of first word in the split
frontsplit = file.columns.str.split('_').str[0]
years = []
#loop to take the year values out of split array
for element in frontsplit:
    if element != 'Formal':
        if element != 'Label':
            years.append(element)
#Printing array to show years being looked at
print(years)


labels = []
for label in file["Label_Name"]:
    labels.append(label)
labels.append("State")
labels.append("County")
print(labels)

['2014', '2015', '2016', '2017']
['Households', 'Households (Family)', 'Households (Non-Family)', 'Average Household Size', 'Average Family Size', 'Number of Higher Ed Enrolled', 'Education of Some College - No Degree Attained', "Education of Associate's Degree Attained", "Education of Bachelor's Degree Attained", 'Education of Graduate or Professional Degree Attained', 'Median Household Income (in dollars)', 'Mean Household Income (in dollars)', 'Median Family Income (in dollars)', 'Mean Family Income (in dollars)', 'Median Non-Family Income (in dollars)', 'Mean Non-Family Income (in dollars)', 'Age of 20 to 24 years', 'Age of 25 to 34 years', 'Age of 35 to 44 years', 'Age of 45 to 54 years', 'Age of 55 to 59 years', 'Age of 60 to 64 years', 'Age of 65 to 74 years', 'Age of 75 to 84 years', 'Age of 85 years and over', 'Race - White', 'Race - Black or African American', 'Race - American Indian and Alaska Native', 'Race - Asian', 'Race - Native Hawaiian and Other Pacific Islander', 'Rac

In [20]:
#Begin For-Loops & Pulling APIs
response = {}
counter = 0
for year in years:
    url = baseurl + year + "/acs/acs1/profile?get="
    
    #building path to pull api with proper csv column
    filename = year + "_Variable"
    
    for row in file[filename]:
        url = url + row + ","
    
    #deletes trailing comma from the above loop
    url = url[:-1]
    url = url + suffix    
    
    response[counter] = requests.get(url).json()
    counter += 1

In [21]:
#Initialize dictionary to hold response data
data = {}
counter = 0
#loop to organize data into dictionary
for year in years:
    data[year] = response[counter][1]
    counter += 1

#Put data into dataframe    
df = pd.DataFrame.from_dict(data, orient='index', columns= labels)
df = df.drop(['State', 'County'], axis=1)
df

,Households,Households (Family),Households (Non-Family),Average Household Size,Average Family Size,Number of Higher Ed Enrolled,Education of Some College - No Degree Attained,Education of Associate's Degree Attained,Education of Bachelor's Degree Attained,Education of Graduate or Professional Degree Attained,...,Age of 75 to 84 years,Age of 85 years and over,Race - White,Race - Black or African American,Race - American Indian and Alaska Native,Race - Asian,Race - Native Hawaiian and Other Pacific Islander,Race - Other,Race - Two or More,Race - Hispanic or Latino
2014,444543,283682,160861,2.75,3.40,122968,161727,95527,170560,88350,...,38027,17620,819441,260845,2682,64590,623,66597,38223,1253001
2015,457736,301978,155758,2.74,3.30,113112,172427,89449,185472,88758,...,37881,19316,829874,263369,2783,68409,1617,85157,36917,1288126
2016,468515,301665,166850,2.74,3.34,130436,161389,90715,196945,97948,...,41494,16964,851713,276302,1697,70844,50,67741,46020,1314367
2017,460602,303666,156936,2.86,3.47,117462,169812,114043,206650,101612,...,43134,18925,831468,286324,2743,69335,1623,107587,49895,1348975


In [23]:
#Output to .csv for future reference
outputfile = "../Resources/demographics.csv"
df.to_csv(outputfile, index_label="Year")